In [104]:
import os
import config
v1, v2, v3 = config.spark_env()
os.environ['SPARK_VERSION'] = v1
os.environ['JAVA_HOME'] = v2
os.environ['SPARK_HOME'] = v3
import findspark
import pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql.functions import array, col, explode, lit, struct, log
from pyspark.sql import DataFrame
from typing import Iterable
import numpy as np
import spark_functions
import tensorflow as tf
import keras.metrics
import pyspark.sql.functions as F
import math
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import seaborn as sns
rds_string = config.rds_string
engine = create_engine(f'postgresql://{rds_string}')
conn = engine.connect()
metadata = MetaData(engine)

In [2]:
india_spark = spark_functions.merged_df("India")

In [3]:
china_spark = spark_functions.merged_df("China")

In [4]:
germany_spark = spark_functions.merged_df("Germany")

In [5]:
france_spark = spark_functions.merged_df("France")

In [6]:
mexico_spark = spark_functions.merged_df("Mexico")

In [7]:
us_gdp = spark_functions.sql_to_spark_gdp("United States")

In [8]:
us_emissions = spark_functions.sql_to_spark_emissions("United States")

In [9]:
us_export, us_import = spark_functions.sql_to_spark_trade("USA")

In [10]:
us_spark = spark_functions.merge_sparks(us_import,us_export,us_gdp,us_emissions)

In [95]:
us_spark.show()

+-------------+----+-----------+---------------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+
|      country|year|        gdp|annual_co2_emissions_tonnes|import_trade_sum_usd|import_weight_sum_kg|import_quantity_sum|export_trade_sum_usd|export_weight_sum_kg|export_quantity_sum|
+-------------+----+-----------+---------------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+
|United States|1991| 24342.2589|                5.0708394E9|        553057797707|     4.2818915615E10|    5.0568428046E10|        507650351345|    1.02364559839E11|   3.90458764909E11|
|United States|1992|25418.99078|              5.174059076E9|        601355275322|     4.3061290517E10|     5.448525663E10|        538062351125|    1.28601940879E11|   3.60206508634E11|
|United States|1993|26387.29373|              5.284687882E9|        6523739

In [116]:
df = us_spark
df = df.withColumn("annual_co2_emissions_tonnes_log", F.log10(col("annual_co2_emissions_tonnes")))
df = df.withColumn("import_trade_sum_usd_log", F.log10(col("import_trade_sum_usd")))
df = df.withColumn("import_weight_sum_kg_log", F.log10(col("import_weight_sum_kg")))
df = df.withColumn("import_quantity_sum_log", F.log10(col("import_quantity_sum")))
df = df.withColumn("export_trade_sum_usd_log", F.log10(col("export_trade_sum_usd")))
df = df.withColumn("export_weight_sum_kg_log", F.log10(col("export_weight_sum_kg")))
df = df.withColumn("export_quantity_sum_log", F.log10(col("export_quantity_sum")))



In [117]:
df.columns

['country',
 'year',
 'gdp',
 'annual_co2_emissions_tonnes',
 'import_trade_sum_usd',
 'import_weight_sum_kg',
 'import_quantity_sum',
 'export_trade_sum_usd',
 'export_weight_sum_kg',
 'export_quantity_sum',
 'annual_co2_emissions_tonnes_log',
 'import_trade_sum_usd_log',
 'import_weight_sum_kg_log',
 'import_quantity_sum_log',
 'export_trade_sum_usd_log',
 'export_weight_sum_kg_log',
 'export_quantity_sum_log']

In [118]:
df.select('country',
 'year',
 'gdp',
 'annual_co2_emissions_tonnes_log',
 'import_trade_sum_usd_log',
 'import_weight_sum_kg_log',
 'import_quantity_sum_log',
 'export_trade_sum_usd_log',
 'export_weight_sum_kg_log',
 'export_quantity_sum_log').show()

+-------------+----+-----------+-------------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+-----------------------+
|      country|year|        gdp|annual_co2_emissions_tonnes_log|import_trade_sum_usd_log|import_weight_sum_kg_log|import_quantity_sum_log|export_trade_sum_usd_log|export_weight_sum_kg_log|export_quantity_sum_log|
+-------------+----+-----------+-------------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+-----------------------+
|United States|1991| 24342.2589|              9.705079856101358|      11.742770519936814|      10.631635664630886|     10.703879453502847|      11.705564691095635|      11.010149623334812|     11.591575176195237|
|United States|1992|25418.99078|              9.713831383065168|      11.779131125122758|      10.634087041186806|     10.736279000800337|      11.7

In [119]:
x_cols = [
 'gdp',
 'import_trade_sum_usd_log',
 'import_weight_sum_kg_log',
 'import_quantity_sum_log',
 'export_trade_sum_usd_log',
 'export_weight_sum_kg_log',
 'export_quantity_sum_log']

In [120]:
arr = np.array(df.select(
 'gdp',
 'import_trade_sum_usd_log',
 'import_weight_sum_kg_log',
 'import_quantity_sum_log',
 'export_trade_sum_usd_log',
 'export_weight_sum_kg_log',
 'export_quantity_sum_log') \
.collect())

In [122]:
arr2 = np.array(df.select('annual_co2_emissions_tonnes_log').collect())

In [123]:
arr

array([[2.43422589e+04, 1.17427705e+01, 1.06316357e+01, 1.07038795e+01,
        1.17055647e+01, 1.10101496e+01, 1.15915752e+01],
       [2.54189908e+04, 1.17791311e+01, 1.06340870e+01, 1.07362790e+01,
        1.17308326e+01, 1.11092475e+01, 1.15565516e+01],
       [2.63872937e+04, 1.18144966e+01, 1.06829451e+01, 1.08639808e+01,
        1.17390966e+01, 1.10958860e+01, 1.15145622e+01],
       [2.76948534e+04, 1.18708430e+01, 1.07182401e+01, 1.07644034e+01,
        1.17769918e+01, 1.10625927e+01, 1.15506953e+01],
       [2.86908757e+04, 1.19192139e+01, 1.07197897e+01, 1.07674525e+01,
        1.18286052e+01, 1.11719079e+01, 1.15916662e+01],
       [2.99677127e+04, 1.19457571e+01, 1.07375898e+01, 1.08271680e+01,
        1.18592284e+01, 1.11269201e+01, 1.16041685e+01],
       [3.14591390e+04, 1.19870048e+01, 1.07660908e+01, 1.08344230e+01,
        1.18999347e+01, 1.10804689e+01, 1.15551031e+01],
       [3.28536770e+04, 1.20118059e+01, 1.07526860e+01, 1.08414360e+01,
        1.19017292e+01, 1

In [124]:
arr2

array([[9.70507986],
       [9.71383138],
       [9.72301934],
       [9.73061976],
       [9.73553609],
       [9.75021414],
       [9.7561712 ],
       [9.75984335],
       [9.76583615],
       [9.77819512],
       [9.77103268],
       [9.77405332],
       [9.77752248],
       [9.78580095],
       [9.78760255],
       [9.78191588],
       [9.78753015],
       [9.7732579 ],
       [9.73999891],
       [9.75595682],
       [9.74591028],
       [9.72970977],
       [9.74190859],
       [9.74575844],
       [9.73406371],
       [9.72516497]])

In [125]:
arr.shape

(26, 7)

In [126]:
arr2.shape

(26, 1)

In [23]:
from sklearn.model_selection import train_test_split


In [127]:
X_train, X_test, y_train, y_test = train_test_split(arr, arr2)

In [128]:
X_scaler = StandardScaler().fit(X_train)

In [129]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# def build_and_compile_model(norm):
#     model = keras.Sequential([
#       norm,
#       layers.Dense(64, activation='relu'),
#       layers.Dense(64, activation='relu'),
#       layers.Dense(1)
#     ])

#     model.compile(loss='mean_absolute_error',
#                 optimizer=tf.keras.optimizers.Adam(0.001))
#     return model

In [130]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=7))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='ReLU'))
model.add(Dense(units=100, activation='PReLU'))
model.add(Dense(units=100, activation='swish'))
model.add(Dense(units=1, activation='linear'))

In [131]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 100)               800       
_________________________________________________________________
dense_28 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_29 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_30 (Dense)             (None, 100)               10200     
_________________________________________________________________
dense_31 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 101       
Total params: 41,401
Trainable params: 41,401
Non-trainable params: 0
__________________________________________________

In [132]:
METRICS = [
      keras.metrics.MeanAbsoluteError(name='mean_absolute_error'),
      keras.metrics.MeanSquaredError(name='mean_squared_error')
]


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='mse', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

EPOCHS = 100

In [133]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=METRICS)

In [134]:
model_history = model.fit(
    X_train_scaled,
    y_train,
    epochs=EPOCHS,
    validation_data=(X_test_scaled, y_test)
)

Epoch 1/100
1/1 [==============================] - 0s 396ms/step - loss: 94.3043 - mean_absolute_error: 9.7109 - mean_squared_error: 94.3043 - val_loss: 92.9882 - val_mean_absolute_error: 9.6429 - val_mean_squared_error: 92.9882
Epoch 2/100
1/1 [==============================] - 0s 19ms/step - loss: 92.5885 - mean_absolute_error: 9.6219 - mean_squared_error: 92.5885 - val_loss: 91.2314 - val_mean_absolute_error: 9.5514 - val_mean_squared_error: 91.2314
Epoch 3/100
1/1 [==============================] - 0s 20ms/step - loss: 90.7103 - mean_absolute_error: 9.5232 - mean_squared_error: 90.7103 - val_loss: 89.2184 - val_mean_absolute_error: 9.4453 - val_mean_squared_error: 89.2184
Epoch 4/100
1/1 [==============================] - 0s 20ms/step - loss: 88.5833 - mean_absolute_error: 9.4099 - mean_squared_error: 88.5833 - val_loss: 86.8422 - val_mean_absolute_error: 9.3185 - val_mean_squared_error: 86.8422
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 86.1122 - mean_

1/1 [==============================] - 0s 22ms/step - loss: 4.1234 - mean_absolute_error: 1.7114 - mean_squared_error: 4.1234 - val_loss: 1.8615 - val_mean_absolute_error: 1.1043 - val_mean_squared_error: 1.8615
Epoch 38/100
1/1 [==============================] - 0s 21ms/step - loss: 3.7379 - mean_absolute_error: 1.6399 - mean_squared_error: 3.7379 - val_loss: 1.9996 - val_mean_absolute_error: 1.0945 - val_mean_squared_error: 1.9996
Epoch 39/100
1/1 [==============================] - 0s 20ms/step - loss: 3.5280 - mean_absolute_error: 1.5777 - mean_squared_error: 3.5280 - val_loss: 2.1755 - val_mean_absolute_error: 1.1164 - val_mean_squared_error: 2.1755
Epoch 40/100
1/1 [==============================] - 0s 20ms/step - loss: 3.4308 - mean_absolute_error: 1.5335 - mean_squared_error: 3.4308 - val_loss: 2.2727 - val_mean_absolute_error: 1.1670 - val_mean_squared_error: 2.2727
Epoch 41/100
1/1 [==============================] - 0s 21ms/step - loss: 3.3570 - mean_absolute_error: 1.4959 - m

1/1 [==============================] - 0s 20ms/step - loss: 0.6423 - mean_absolute_error: 0.6017 - mean_squared_error: 0.6423 - val_loss: 0.4117 - val_mean_absolute_error: 0.5552 - val_mean_squared_error: 0.4117
Epoch 74/100
1/1 [==============================] - 0s 20ms/step - loss: 0.6214 - mean_absolute_error: 0.5968 - mean_squared_error: 0.6214 - val_loss: 0.3795 - val_mean_absolute_error: 0.5308 - val_mean_squared_error: 0.3795
Epoch 75/100
1/1 [==============================] - 0s 21ms/step - loss: 0.5983 - mean_absolute_error: 0.5878 - mean_squared_error: 0.5983 - val_loss: 0.3551 - val_mean_absolute_error: 0.5116 - val_mean_squared_error: 0.3551
Epoch 76/100
1/1 [==============================] - 0s 22ms/step - loss: 0.5699 - mean_absolute_error: 0.5744 - mean_squared_error: 0.5699 - val_loss: 0.3372 - val_mean_absolute_error: 0.4907 - val_mean_squared_error: 0.3372
Epoch 77/100
1/1 [==============================] - 0s 21ms/step - loss: 0.5379 - mean_absolute_error: 0.5569 - m

In [136]:
predictions = model.predict(X_test_scaled)

In [137]:
predictions[0][0]

9.921507

In [138]:
y_test[0][0]

9.756171195425988

In [139]:
10**predictions[0][0]

8346547738.882024

In [54]:
X_scaler.inverse_transform(predictions)

ValueError: non-broadcastable output operand with shape (7,1) doesn't match the broadcast shape (7,7)

In [ ]:

# plot_cm(title,file_save,labels, predictions, p=0.5):
cm = confusion_matrix(labels, predictions)
plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt="d")
plt.title(title)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.savefig(file_save, dpi=300)